In [4]:
#Imprort des librairies nécessaires
import pandas as pd
import numpy as np

In [5]:
#Preparation des jeux de données autobus:

#1)Lecture du fichier
excel_autobus = pd.read_excel("https://www.statistiques.developpement-durable.gouv.fr/sites/default/files/2019-04/2019-parc-vehicules-routiers-3pf2.xls", header = None)

#2)Division du jeu de do en 2
excel_autobus_ap_2010= excel_autobus[:66]
excel_autobus_av_2010= excel_autobus[66:]

#3)Nom de colonnes :
col_names_ap_2010_temp=["Classes de places assises","2010","2011","2012","2013","2014","2015","2016","2017","2018","2019"]
col_names_av_2010_temp=["Classes de places assises","2001","2002","2003","2004","2005","2006","2007","2008","2009"]

###4.1) Formatage dataset - Après 2010: 
excel_autobus_ap_2010.columns = col_names_ap_2010_temp
#Supp ds 3 premières lignes et de la derniere (afin de garder un ligne vide avant et après)
excel_autobus_ap_2010=excel_autobus_ap_2010[3:]
excel_autobus_ap_2010=excel_autobus_ap_2010[:-1]
excel_autobus_ap_2010=excel_autobus_ap_2010.reset_index(drop=True)

###4.2) Formatage dataset - Avant 2010:
#Suppression de la dernière colonne et attribution de noms de colonnes
excel_autobus_av_2010=excel_autobus_av_2010.iloc[:, :-1]
excel_autobus_av_2010.columns=col_names_av_2010_temp

#Supp de la première ligne et rajout d'une derniere ligne (afin de garder un ligne vide avant et après)
excel_autobus_av_2010.loc[len(excel_autobus_av_2010)] = np.nan
excel_autobus_av_2010=excel_autobus_av_2010[1:]
excel_autobus_av_2010=excel_autobus_av_2010.reset_index(drop=True)

In [6]:
#Preparation des jeux de données cam_camionette:

#1)Lecture du fichier
excel_cam = pd.read_excel("https://www.statistiques.developpement-durable.gouv.fr/sites/default/files/2019-04/2019-parc-vehicules-routiers-5pf2.xls", header = None)

#2)Nom de colonnes :
col_names_cam=["Classes de PTAC","2010","2011","2012","2013","2014","2015","2016","2017","2018","2019"]

#3) Formatage Dataset
#Attribution des noms de colonnes
excel_cam.columns = col_names_cam
#Supp ds 2 premières lignes et de la derniere (afin de garder un ligne vide avant et après)
excel_cam=excel_cam[2:]
excel_cam=excel_cam[:-1]
excel_cam=excel_cam.reset_index(drop=True)
excel_cam.iloc[0]=np.nan # On set toute la première ligne à null

In [7]:
def recup_tab_energie (dataset, tab_colnames):
    col_categorie=tab_colnames[0]
    #Récuperation des lignes de séparation entre chaque tableau
    sep=[l for l in range(len(dataset)) if dataset.iloc[l].isna().all()]

    #Récuperation des tableaux par energie
    tab=dict()
    for s in range(len(sep)-1) :
        #print(sep[s]+1, dataset['Classes de places assises'][sep[s]+1])
        tab[dataset[col_categorie][sep[s]+1]]=dataset.iloc[sep[s]+3:sep[s+1]-1]
        #sep[s]+3 afin d'enlever la ligne vide + la ligne du type denergie + la ligne des titres de colonnes
        #sep[s+1]-1 afin d'enlever la ligne Total
    return (tab)
    

In [8]:
def pivot_and_merge (tab, tab_colnames):
    col_categorie=tab_colnames[0]
    #Pivot de chaque tableau et récupération sous forme de liste
    tab_transfo=list()
    for energie in tab:
        #tab[energie]=pd.melt(tab[energie],id_vars=[col_categorie], value_vars=["2010","2011","2012","2013","2014","2015","2016","2017","2018","2019"],var_name="Année", value_name=str(energie))
        #tab_transfo=list(tab.values())
        tab_transfo.append(pd.melt(tab[energie],id_vars=[col_categorie], value_vars=tab_colnames[1:],var_name="Année", value_name=str(energie)))

    # Merging des tableaux
    tab_transfo_merged=tab_transfo[0]
    del tab_transfo[0]
    for tab in tab_transfo:
        tab_transfo_merged = pd.merge(tab_transfo_merged, tab,on=[col_categorie,'Année'],how='outer')

    #Remplacement des valeurs nulles
    tab_transfo_merged.iloc[:,2:]=tab_transfo_merged.iloc[:,2:].fillna(0).astype(int)
    #tab_transfo_merged.isnull().sum()
    
    return(tab_transfo_merged)
   

In [21]:
autobus_ap_2010_transfo=pivot_and_merge(recup_tab_energie(excel_autobus_ap_2010,col_names_ap_2010_temp),col_names_ap_2010_temp)

autobus_ap_2010_transfo.to_csv("autobus_ap_2010_par_type.csv",sep=';', index = False)

autobus_ap_2010_transfo_v2= pd.melt(autobus_ap_2010_transfo , id_vars=['Classes de places assises', 'Année'], value_vars=['Gazole', 'Gaz naturel','Gazole-électricité', 'Autres hydrocarbures gazeux comprimés', 'Electricité', 'Autres énergies'], value_name="Nb_véhicules", var_name="Fuel")

autobus_ap_2010_transfo_v2.to_csv("autobus_ap_2010_par_type_v2.csv",sep=';', index = False)



autobus_av_2010_transfo=pivot_and_merge(recup_tab_energie(excel_autobus_av_2010,col_names_av_2010_temp),col_names_av_2010_temp)

autobus_av_2010_transfo.to_csv("autobus_av_2010_par_type.csv",sep=';', index = False)

autobus_av_2010_transfo_v2= pd.melt(autobus_av_2010_transfo , id_vars=['Classes de places assises', 'Année'], value_vars=['Essence (ordinaire et super)','Gazole', 'Gaz naturel véhicule (GNV)', 'Autres  et  non déterminées'], value_name="Nb_véhicules", var_name="Fuel")

autobus_av_2010_transfo_v2.to_csv("autobus_av_2010_par_type_v2.csv",sep=';', index = False)


In [10]:
tab_cam=recup_tab_energie(excel_cam,col_names_cam) 

tab_camionnette=dict()
tab_camion=dict()

#Recupération des tableau camion d'un coté et camionettes de l'autre
for energie in tab_cam:
    tab_cam[energie]=tab_cam[energie].reset_index(drop=True)
    
    #sep_cam_camion[energie]= [l for l in range(len(tab_cam[energie])) if "Total" in tab_cam[energie]["Classes de PTAC"][l]]
    
    tab_camionnette[energie]=tab_cam[energie][0:4]
    tab_camion[energie]=tab_cam[energie][5:12]

tab_camion_transfo=pivot_and_merge(tab_camion, col_names_cam)

#tab_camion_transfo.to_csv("camion_par_type.csv",sep=';', index = False)

tab_camion_transfo_v2 = pd.melt(tab_camion_transfo , id_vars=['Classes de PTAC', 'Année'], value_vars=['Gazole', 'Essence', 'Electricité', 'Essence-GPL', 'Autres et non  déterminées'], value_name="Nb_véhicules", var_name="Fuel")

tab_camion_transfo_v2.to_csv("camion_par_type_v2.csv",sep=';', index = False)


tab_camionnette_transfo=pivot_and_merge(tab_camionnette, col_names_cam)

#tab_camionnette_transfo.to_csv("camionnette_par_type.csv",sep=';', index = False)

tab_camionnette_transfo_v2=pd.melt(tab_camionnette_transfo , id_vars=['Classes de PTAC', 'Année'], value_vars=['Gazole', 'Essence', 'Electricité', 'Essence-GPL', 'Autres et non  déterminées'], value_name="Nb_véhicules", var_name="Fuel")

tab_camionnette_transfo_v2.to_csv("camionnette_par_type_v2.csv",sep=';', index = False)


In [26]:
tab_camionnette_transfo.columns

Index(['Classes de PTAC', 'Année', 'Gazole', 'Essence', 'Electricité',
       'Essence-GPL', 'Autres et non  déterminées'],
      dtype='object')

In [27]:
tab_camion_transfo.columns

Index(['Classes de PTAC', 'Année', 'Gazole', 'Essence', 'Electricité',
       'Essence-GPL', 'Autres et non  déterminées'],
      dtype='object')

In [28]:
pd.melt(tab_camionnette_transfo , id_vars=['Classes de PTAC', 'Année'], value_vars=['Gazole', 'Essence', 'Electricité', 'Essence-GPL', 'Autres et non  déterminées'], value_name="Nb_véhicules", var_name="Fuel")


,Classes de PTAC,Année,Fuel,Nb_véhicules
0,"Moins de 1,5 tonnes",2010,Gazole,765442
1,"1,5 à 2,5 tonnes",2010,Gazole,2959183
2,"2,6 à 3,5 tonnes",2010,Gazole,2084076
3,Non déterminée,2010,Gazole,509
4,"Moins de 1,5 tonnes",2011,Gazole,663230
...,...,...,...,...
195,Non déterminée,2018,Autres et non déterminées,42
196,"Moins de 1,5 tonnes",2019,Autres et non déterminées,1953
197,"1,5 à 2,5 tonnes",2019,Autres et non déterminées,6073
198,"2,6 à 3,5 tonnes",2019,Autres et non déterminées,3038
